# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [3]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [4]:
# Read in data
df = pd.read_csv("zillow_cleaned.csv")

In [5]:
# Features and target
X = df.drop(columns='taxvaluedollarcnt')
y = df['taxvaluedollarcnt']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [6]:
X_train.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,poolcnt,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,numberofstories
75034,11010994,3.0,4.0,2286.0,2286.0,6037.0,0.0,0.0,0.0,17199.0,0.0,261.0,6.037107e+07,12447.0,3101.0,96368.0,0.0,0.0
42599,11407829,1.0,2.0,864.0,864.0,6037.0,0.0,0.0,0.0,3430.0,0.0,261.0,6.037601e+07,45888.0,3101.0,96133.0,0.0,0.0
71027,12302592,2.0,4.0,1556.0,1515.0,6037.0,0.0,0.0,0.0,3664.0,0.0,246.0,6.037536e+07,32616.0,3101.0,96125.0,0.0,0.0
62724,17290835,2.5,3.0,1949.0,1949.0,6111.0,1.0,2.0,0.0,7200.0,0.0,266.0,6.111007e+07,34278.0,2061.0,96385.0,7.0,2.0
7790,14649487,2.5,2.0,1386.0,1386.0,6059.0,0.0,0.0,0.0,7200.0,0.0,266.0,6.059110e+07,24832.0,1286.0,97052.0,6.0,1.0


In [7]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler **only on training data**
X_train_scaled = scaler.fit_transform(X_train)

X_train_scaled

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_train_scaled

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,poolcnt,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,numberofstories
75034,-0.582844,0.763780,0.849543,0.628465,0.697315,-0.569744,-0.316933,-0.654129,-0.519005,-0.087355,-0.507892,-0.158844,-0.583968,-0.441944,0.707220,-0.061669,-0.523241,-0.497515
42599,-0.466933,-1.343706,-0.922483,-1.043632,-1.044933,-0.569744,-0.316933,-0.654129,-0.519005,-0.202328,-0.507892,-0.158844,-0.559966,0.259868,0.707220,-0.129134,-0.523241,-0.497515
71027,-0.205584,-0.289963,0.849543,-0.229925,-0.247321,-0.569744,-0.316933,-0.654129,-0.519005,-0.200374,-0.507892,-3.014131,-0.563149,-0.018665,0.707220,-0.131431,-0.523241,-0.497515
62724,1.251421,0.236908,-0.036470,0.232194,0.284420,2.993375,2.206741,1.546033,-0.519005,-0.170848,-0.507892,0.792918,3.001574,0.016214,-0.590409,-0.056789,1.960004,2.556213
7790,0.479915,0.236908,-0.922483,-0.429825,-0.405373,0.489562,-0.316933,-0.654129,-0.519005,-0.170848,-0.507892,0.792918,0.483606,-0.182025,-1.557392,0.134698,1.605255,1.029349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,-0.566046,0.763780,0.849543,1.359860,1.459396,-0.569744,-0.316933,-0.654129,-0.519005,-0.145147,-0.507892,1.363975,-0.583886,-0.441944,0.707220,-0.065114,-0.523241,-0.497515
6265,1.186637,0.236908,0.849543,0.047581,0.092062,2.993375,4.730414,1.546033,1.666976,-0.178781,-0.507892,-0.158844,3.001278,0.138608,-0.590409,0.147904,1.605255,1.029349
54886,-0.156372,-0.289963,-0.036470,-0.010036,0.032027,-0.569744,-0.316933,-0.654129,-0.519005,-0.188634,-0.507892,-0.158844,-0.561417,-0.445197,0.707220,-0.106455,-0.523241,-0.497515
860,-0.607221,-1.343706,-0.922483,-0.676759,-0.662667,-0.569744,-0.316933,-0.654129,-0.519005,-0.184760,-0.507892,-0.158844,-0.583105,-0.441944,0.707220,-0.039276,-0.523241,-0.497515


### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [8]:
# RMSE scorer (negative because sklearn expects higher = better)
from sklearn.metrics import make_scorer
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [9]:
from sklearn.tree import DecisionTreeRegressor
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Bagging": BaggingRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

In [15]:
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
results = []

for name, model in models.items():
    # Negative RMSE, so we multiply by -1 to get actual RMSE
    scores = cross_val_score(model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
    results.append({
        "Model": name,
        "Mean RMSE": np.mean(scores),
        "Std RMSE": np.std(scores)
    })

results_df = pd.DataFrame(results).sort_values("Mean RMSE")


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.138e+15, tolerance: 7.053e+11
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.113e+15, tolerance: 6.955e+11
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

In [16]:
results_df

,Model,Mean RMSE,Std RMSE
3,Decision Tree,-348812.770651,5624.380511
2,Lasso Regression,-294780.081287,3092.177989
1,Ridge Regression,-293445.942543,3087.588510
0,Linear Regression,-293276.018549,3082.936162
4,Bagging,-265998.969586,3166.359911
6,Gradient Boosting,-259281.291346,2758.551127
5,Random Forest,-255558.279812,2686.459820


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> Your text here

The Random Forest model performed the best overall, with the lowest average RMSE, meaning it made the most accurate predictions. It was also the most consistent, with the smallest variation in scores across different cross-validation folds. Gradient Boosting also performed well, coming in second in both accuracy and stability. In contrast, the Decision Tree had the highest RMSE and more variation, which could mean it's overfitting. The Linear, Ridge, and Lasso regression models all had similar and very stable results, but their higher RMSE scores suggest they may be underfitting.

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [10]:
# Log of finishedsquarefeet12 (to reduce skew)
X_train["log_finishedsqft"] = np.log1p(X_train["finishedsquarefeet12"])
# Interaction term (bathroomcnt * bedroomcnt)
X_train["bath_bed_interaction"] = X_train["bathroomcnt"] * X_train["bedroomcnt"]
# Ratio of finished sqft to lot size (space utilization)
X_train["sqft_to_lot_ratio"] = X_train["finishedsquarefeet12"] / X_train["lotsizesquarefeet"]

# Drop the original column
X_train = X_train.drop(columns=["finishedsquarefeet12"])

# Drop any inf or NaN values created by division
X_train = X_train.replace([np.inf, -np.inf], np.nan).dropna()
y_train = y_train.loc[X_train.index]  # align target with cleaned data

# === Step 2: Standardize ===
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
results = []

for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, scoring=rmse_scorer, cv=cv)
    results.append({
        "Model": name,
        "Mean RMSE": -np.mean(scores),  # Flip sign back to positive
        "Std RMSE": np.std(scores)
    })

results_df = pd.DataFrame(results).sort_values("Mean RMSE")

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.152e+15, tolerance: 7.053e+11
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.125e+15, tolerance: 6.955e+11
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

In [1]:
results_df

NameError: name 'results_df' is not defined

### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



> Your text here

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [5]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here